In [109]:
import pandas as pd

In [110]:
import numpy as np

In [111]:
import wikipedia as wp

In [112]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [113]:
import json
import lxml.html as lh
import bs4 as bs
import urllib.request

In [114]:
from geopy.geocoders import Nominatim

In [115]:
import requests
from pandas.io.json import json_normalize

In [116]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [117]:
from sklearn.cluster import KMeans

In [118]:
import folium

In [151]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df


Postal code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
7           M8A      Not assigned   
8           M9A         Etobicoke   
9           M1B       Scarborough   
10          M2B      Not assigned   
11          M3B        North York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M6B        North York   
15          M7B      Not assigned   
16          M8B      Not assigned   
17          M9B         Etobicoke   
18          M1C       Scarborough   
19          M2C      Not assigned   
20          M3C        North York   
21          M4C         East York   
22          M5C  Downtown Toronto   
23          M6C              York   
24          M7C      Not assigned   
25          M8C      Not assigned   
26          M9C         Etobicoke   
27          M1E       Scarborough   
28          M2E      Not assigned   
29          M3E      Not assigned   
30          M4E      East Toronto   
31          M5E  Downtown Toronto   
32          M6E              York   
33          M7E      Not assigned   
34          M8E      Not assigned   
35          M9E      Not assigned   
36          M1G       Scarborough   
37          M2G      Not assigned   
38          M3G      Not assigned   
39          M4G         East York   
40          M5G  Downtown Toronto   
41          M6G  Downtown Toronto   
42          M7G      Not assigned   
43          M8G      Not assigned   
44          M9G      Not assigned   
45          M1H       Scarborough   
46          M2H        North York   
47          M3H        North York   
48          M4H         East York   
49          M5H  Downtown Toronto   
50          M6H      West Toronto   
51          M7H      Not assigned   
52          M8H      Not assigned   
53          M9H      Not assigned   
54          M1J       Scarborough   
55          M2J        North York   
56          M3J        North York   
57          M4J         East York   
58          M5J  Downtown Toronto   
59          M6J      West Toronto   
60          M7J      Not assigned   
61          M8J      Not assigned   
62          M9J      Not assigned   
63          M1K       Scarborough   
64          M2K        North York   
65          M3K        North York   
66          M4K      East Toronto   
67          M5K  Downtown Toronto   
68          M6K      West Toronto   
69          M7K      Not assigned   
70          M8K      Not assigned   
71          M9K      Not assigned   
72          M1L       Scarborough   
73          M2L        North York   
74          M3L        North York   
75          M4L      East Toronto   
76          M5L  Downtown Toronto   
77          M6L        North York   
78          M7L      Not assigned   
79          M8L      Not assigned   
80          M9L        North York   
81          M1M       Scarborough   
82          M2M        North York   
83          M3M        North York   
84          M4M      East Toronto   
85          M5M        North York   
86          M6M              York   
87          M7M      Not assigned   
88          M8M      Not assigned   
89          M9M        North York   
90          M1N       Scarborough   
91          M2N        North York   
92          M3N        North York   
93          M4N   Central Toronto   
94          M5N   Central Toronto   
95          M6N              York   
96          M7N      Not assigned   
97          M8N      Not assigned   
98          M9N              York   
99          M1P       Scarborough   
100         M2P        North York   
101         M3P      Not assigned   
102         M4P   Central Toronto   
103         M5P   Central Toronto   
104         M6P      West Toronto   
105         M7P      Not assigned   
106         M8P      Not assigned   
107     

In [152]:

#Delete rows where Borough is 'Not assigned'
df = df[df.Borough!= "Not assigned"]

#Combine all neighbourhoods where postcode and Borough are the same
df = df.groupby(['Postal code', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
#if Neighbourhood is not assigned, replace by borough name
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

# define the dataframe columns
column_names = ['Postal code','Borough', 'Neighborhood'] 

# instantiate the dataframe
df.columns = column_names

df.head()

Postal code      Borough                              Neighborhood
0         M1B  Scarborough                           Malvern / Rouge
1         M1C  Scarborough  Rouge Hill / Port Union / Highland Creek
2         M1E  Scarborough       Guildwood / Morningside / West Hill
3         M1G  Scarborough                                    Woburn
4         M1H  Scarborough                                 Cedarbrae

In [155]:
import io
import requests
url="https://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
dfc = df.join(c.set_index('Postal Code'), on='Postal code')
dfc.head()


Postal code      Borough                              Neighborhood  \
0         M1B  Scarborough                           Malvern / Rouge   
1         M1C  Scarborough  Rouge Hill / Port Union / Highland Creek   
2         M1E  Scarborough       Guildwood / Morningside / West Hill   
3         M1G  Scarborough                                    Woburn   
4         M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476

In [156]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [157]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfc['Latitude'], dfc['Longitude'], dfc['Borough'], dfc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [158]:
CLIENT_ID = 'Z44CVA5LDCOAB3BZ1ZZKIKXEWUHMAJO32F45PGZ4WF02TEZF' # your Foursquare ID
CLIENT_SECRET = 'K0J0JWODJRGP4W4V44EUIG1Q2O2THTVKE4ORUPZDXYQGCH4Q' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius = 500
#print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

Your credentails:
CLIENT_ID: Z44CVA5LDCOAB3BZ1ZZKIKXEWUHMAJO32F45PGZ4WF02TEZF
CLIENT_SECRET:K0J0JWODJRGP4W4V44EUIG1Q2O2THTVKE4ORUPZDXYQGCH4Q


'https://api.foursquare.com/v2/venues/search?client_id=Z44CVA5LDCOAB3BZ1ZZKIKXEWUHMAJO32F45PGZ4WF02TEZF&client_secret=K0J0JWODJRGP4W4V44EUIG1Q2O2THTVKE4ORUPZDXYQGCH4Q&ll=43.6534817,-79.3839347&v=20180604&radius=500&limit=100'

In [159]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [160]:
results = requests.get(url).json()
results

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

id                                          name  \
0  4c093ee0340720a153728493                    City Hall Council Chambers   
1  4ad4c05ef964a5208ff620e3                             Toronto City Hall   
2  50885719498ea7b5aab3a74c  GoodLife Fitness Toronto Bell Trinity Centre   
3  5b193c42598e64002ca79b96       City of Toronto Civic Innovation Office   
4  4b06ef4cf964a52072f322e3                            Cafe On The Square   

                                          categories    referralId  hasPerk  \
0  [{'id': '4bf58dd8d48988d129941735', 'name': 'C...  v-1586996070    False   
1  [{'id': '4bf58dd8d48988d129941735', 'name': 'C...  v-1586996070    False   
2  [{'id': '4bf58dd8d48988d176941735', 'name': 'G...  v-1586996070    False   
3  [{'id': '4bf58dd8d48988d129941735', 'name': 'C...  v-1586996070    False   
4  [{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...  v-1586996070    False   

   location.lat  location.lng  \
0     43.651827    -79.383949   
1     43.653140    -79.383967   
2     43.653436    -79.382314   
3     43.653454    -79.383952   
4     43.652633    -79.383361   

                             location.labeledLatLngs  location.distance  \
0  [{'label': 'display', 'lat': 43.65182710471462...                184   
1  [{'label': 'display', 'lat': 43.65313989695342...                 38   
2  [{'label': 'display', 'lat': 43.653436, 'lng':...                130   
3  [{'label': 'display', 'lat': 43.653454, 'lng':...                  3   
4  [{'label': 'display', 'lat': 43.65263298456729...                105   

  location.cc location.country  \
0          CA           Canada   
1          CA           Canada   
2          CA           Canada   
3          CA           Canada   
4          CA           Canada   

                           location.formattedAddress  location.address  \
0                                           [Canada]               NaN   
1  [100 Queen St. W. (at Bay St.), Toronto ON M5H...  100 Queen St. W.   
2           [483 Bay St, Toronto ON M5G 2C9, Canada]        483 Bay St   
3       [100 Queen St W, Toronto ON M5H 2N2, Canada]    100 Queen St W   
4       [100 Queen St W, Toronto ON M5H 2N2, Canada]    100 Queen St W   

  location.crossStreet location.postalCode location.city location.state  
0                  NaN                 NaN           NaN            NaN  
1           at Bay St.             M5H 2N2       Toronto             ON  
2                  NaN             M5G 2C9       Toronto             ON  
3                  NaN             M5H 2N2       Toronto             ON  
4                  NaN             M5H 2N2       Toronto             ON

In [161]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

name  categories        lat  \
0                    City Hall Council Chambers   City Hall  43.651827   
1                             Toronto City Hall   City Hall  43.653140   
2  GoodLife Fitness Toronto Bell Trinity Centre         Gym  43.653436   
3       City of Toronto Civic Innovation Office   City Hall  43.653454   
4                            Cafe On The Square  Restaurant  43.652633   

         lng                                     labeledLatLngs  distance  cc  \
0 -79.383949  [{'label': 'display', 'lat': 43.65182710471462...       184  CA   
1 -79.383967  [{'label': 'display', 'lat': 43.65313989695342...        38  CA   
2 -79.382314  [{'label': 'display', 'lat': 43.653436, 'lng':...       130  CA   
3 -79.383952  [{'label': 'display', 'lat': 43.653454, 'lng':...         3  CA   
4 -79.383361  [{'label': 'display', 'lat': 43.65263298456729...       105  CA   

  country                                   formattedAddress  \
0  Canada                                           [Canada]   
1  Canada  [100 Queen St. W. (at Bay St.), Toronto ON M5H...   
2  Canada           [483 Bay St, Toronto ON M5G 2C9, Canada]   
3  Canada       [100 Queen St W, Toronto ON M5H 2N2, Canada]   
4  Canada       [100 Queen St W, Toronto ON M5H 2N2, Canada]   

            address crossStreet postalCode     city state  \
0               NaN         NaN        NaN      NaN   NaN   
1  100 Queen St. W.  at Bay St.    M5H 2N2  Toronto    ON   
2        483 Bay St         NaN    M5G 2C9  Toronto    ON   
3    100 Queen St W         NaN    M5H 2N2  Toronto    ON   
4    100 Queen St W         NaN    M5H 2N2  Toronto    ON   

                         id  
0  4c093ee0340720a153728493  
1  4ad4c05ef964a5208ff620e3  
2  50885719498ea7b5aab3a74c  
3  5b193c42598e64002ca79b96  
4  4b06ef4cf964a52072f322e3

In [162]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [163]:
toronto_venues = getNearbyVenues(names=dfc['Neighborhood'],
                                   latitudes=dfc['Latitude'],
                                   longitudes=dfc['Longitude']
                                  )

toronto_venues.head()

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

Neighborhood  Neighborhood Latitude  \
0                           Malvern / Rouge              43.806686   
1                           Malvern / Rouge              43.806686   
2  Rouge Hill / Port Union / Highland Creek              43.784535   
3  Rouge Hill / Port Union / Highland Creek              43.784535   
4  Rouge Hill / Port Union / Highland Creek              43.784535   

   Neighborhood Longitude                                              Venue  \
0              -79.194353                                            Wendy’s   
1              -79.194353                       T Hamilton & Son Roofing Inc   
2              -79.160497                              Royal Canadian Legion   
3              -79.160497                     Scarborough Historical Society   
4              -79.160497  SEBS Engineering Inc. (Sustainable Energy and ...   

   Venue Latitude  Venue Longitude              Venue Category  
0       43.807448       -79.199056        Fast Food Restaurant  
1       43.807985       -79.198194  Construction & Landscaping  
2       43.782533       -79.163085                         Bar  
3       43.788755       -79.162438              History Museum  
4       43.782371       -79.156820  Construction & Landscaping

In [164]:
print('{} venues were returned by Foursquare.'.format(toronto_venues.shape[0]))
toronto_venues.head()

2142 venues were returned by Foursquare.


Neighborhood  Neighborhood Latitude  \
0                           Malvern / Rouge              43.806686   
1                           Malvern / Rouge              43.806686   
2  Rouge Hill / Port Union / Highland Creek              43.784535   
3  Rouge Hill / Port Union / Highland Creek              43.784535   
4  Rouge Hill / Port Union / Highland Creek              43.784535   

   Neighborhood Longitude                                              Venue  \
0              -79.194353                                            Wendy’s   
1              -79.194353                       T Hamilton & Son Roofing Inc   
2              -79.160497                              Royal Canadian Legion   
3              -79.160497                     Scarborough Historical Society   
4              -79.160497  SEBS Engineering Inc. (Sustainable Energy and ...   

   Venue Latitude  Venue Longitude              Venue Category  
0       43.807448       -79.199056        Fast Food Restaurant  
1       43.807985       -79.198194  Construction & Landscaping  
2       43.782533       -79.163085                         Bar  
3       43.788755       -79.162438              History Museum  
4       43.782371       -79.156820  Construction & Landscaping

In [165]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Agincourt                                                               5   
Alderwood / Long Branch                                                 9   
Bathurst Manor / Wilson Heights / Downsview North                      20   
Bayview Village                                                         4   
Bedford Park / Lawrence Manor East                                     24   
Berczy Park                                                            55   
Birch Cliff / Cliffside West                                            4   
Brockton / Parkdale Village / Exhibition Place                         22   
Business reply mail Processing CentrE                                  17   
CN Tower / King and Spadina / Railway Lands / H...                     18   
Caledonia-Fairbanks                                                     4   
Canada Post Gateway Processing Centre                                  13   
Cedarbrae                                                               8   
Central Bay Street                                                     64   
Christie                                                               18   
Church and Wellesley                                                   75   
Clarks Corners / Tam O'Shanter / Sullivan                              12   
Cliffside / Cliffcrest / Scarborough Village West                       3   
Commerce Court / Victoria Hotel                                       100   
Davisville                                                             36   
Davisville North                                                        8   
Del Ray / Mount Dennis / Keelsdale and Silverthorn                      4   
Don Mills                                                              27   
Dorset Park / Wexford Heights / Scarborough Tow...                      6   
Downsview                                                              16   
Dufferin / Dovercourt Village                                          17   
East Toronto                                                            3   
Eringate / Bloordale Gardens / Old Burnhamthorp...                     10   
Fairview / Henry Farm / Oriole                                         65   
First Canadian Place / Underground city                               100   
Forest Hill North & West                                                4   
Garden District, Ryerson                                              100   
Glencairn                                                               5   
Golden Mile / Clairlea / Oakridge                                       8   
Guildwood / Morningside / West Hill                                     7   
Harbourfront East / Union Station / Toronto Isl...                    100   
High Park / The Junction South                                         24   
Hillcrest Village                                                       4   
Humber Summit                                                           1   
Humberlea / Emery                                                       3   
Humewood-Cedarvale                                                      3   
India Bazaar / The Beaches West                                        20   
Kennedy Park / Ionview / East Birchmount Park                           4   
Kensington Market / Chinatown / Grange Park                            62   
Kingsview Village / St. Phillips / Martin Grove...                      4   
Lawrence Manor / Lawrence Heights                                      12   
Lawrence Park                                                           4   
Leaside                                                                33   
Little Portugal / Trinity                                              43   
Malvern / Rouge                                                         2   
Milliken / Agincourt North / Steeles East / L'A.

In [166]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 265 uniques categories.


In [167]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Accessories Store  Airport  Airport Food Court  Airport Gate  \
0                  0        0                   0             0   
1                  0        0                   0             0   
2                  0        0                   0             0   
3                  0        0                   0             0   
4                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Arts & Crafts Store  Asian Restaurant  \
0             0         0            0                    0                 0   
1             0         0            0                    0                 0   
2             0         0            0                    0                 0   
3             0         0            0                    0                 0   
4             0         0            0                    0                 0   

   Athletics & Sports  Auto Garage  Auto Workshop  BBQ Joint  Baby Store  \
0                   0            0              0          0           0   
1                   0            0              0          0           0   
2                   0            0              0          0           0   
3                   0            0              0          0           0   
4                   0            0              0          0           0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Field  Baseball Stadium  \
0           0       0     0    0               0                 0   
1           0       0     0    0               0                 0   
2           0       0     0    1               0                 0   
3           0       0     0    0               0                 0   
4           0       0     0    0               0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  Bookstore  Boutique  \
0                   0          0       0              0          0         0   
1                   0          0       0              0          0         0   
2                   0          0       0              0          0         0   
3                   0          0       0              0          0         0   
4                   0          0       0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bridal Shop  \
0                     0               0        0            0   
1                     0               0        0            0   
2                     0               0        0            0   
3                     0               0        0            0   
4                     0               0        0            0   

   Bubble Tea Shop  Building  Burger Joint  Burrito Place  Bus Line  \
0                0         0             0              0         0   
1                0         0             0              0         0   
2                0         0             0              0         0   
3                0         0             0              0         0   
4                0         0             0              0         0   

   Bus Station  Business Service  Butcher  Caf

In [168]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Accessories Store  \
0                                          Agincourt                0.0   
1                            Alderwood / Long Branch                0.0   
2  Bathurst Manor / Wilson Heights / Downsview North                0.0   
3                                    Bayview Village                0.0   
4                 Bedford Park / Lawrence Manor East                0.0   

   Airport  Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0      0.0                 0.0           0.0             0.0              0.0   
1      0.0                 0.0           0.0             0.0              0.0   
2      0.0                 0.0           0.0             0.0              0.0   
3      0.0                 0.0           0.0             0.0              0.0   
4      0.0                 0.0           0.0             0.0              0.0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0               0.0             0.000000           0.0       0.0          0.0   
1               0.0             0.000000           0.0       0.0          0.0   
2               0.0             0.000000           0.0       0.0          0.0   
3               0.0             0.000000           0.0       0.0          0.0   
4               0.0             0.041667           0.0       0.0          0.0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Garage  \
0                  0.0               0.0                 0.0          0.0   
1                  0.0               0.0                 0.0          0.0   
2                  0.0               0.0                 0.0          0.0   
3                  0.0               0.0                 0.0          0.0   
4                  0.0               0.0                 0.0          0.0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0            0.0        0.0         0.0         0.0     0.0  0.00  0.0   
1            0.0        0.0         0.0         0.0     0.0  0.00  0.0   
2            0.0        0.0         0.0         0.0     0.0  0.10  0.0   
3            0.0        0.0         0.0         0.0     0.0  0.25  0.0   
4            0.0        0.0         0.0         0.0     0.0  0.00  0.0   

   Baseball Field  Baseball Stadium  Basketball Stadium  Beach  \
0             0.0               0.0                 0.0    0.0   
1             0.0               0.0                 0.0    0.0   
2             0.0               0.0                 0.0    0.0   
3             0.0               0.0                 0.0    0.0   
4             0.0               0.0                 0.0    0.0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bike Shop  \
0              0.0       0.0         0.0                 0.0        0.0   
1              0.0       0.0         0.0                 0.0        0.0   
2              0.0       0.0         0.0                 0.0        0.0   
3              0.0       0.0         0.0                 0.0        0.0   
4              0.0       0.0         0.0                 0.0        0.0   

   Bistro  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0     0.0            0.0        0.0  0.000000                   0.0   
1     0.0            0.0        0.0  0.000000                   0.0   
2     0.0            0.0        0.0  0.000000                   0.0   
3     0.0            0.0        0.0  0.000000                   0.0   
4     0.0            0.0        0.0  0.041667                   0.0   

   Breakfast Spot  Brewery  Bridal Shop  Bubble Tea Shop  Building  \
0             0.2      0.0         0.00              0.0       0.0   
1             0.0      0.0         0.00              0.0       0.0   
2             0.0      0.0         0.05              0.0       0.0   
3             0.0      0.0         0.00              0.0       0.0   
4             0.0      0.0         0.00              0.0       0.0   

   Burger J

In [169]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1             Clothing Store   0.2
2                     Lounge   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


----Alderwood / Long Branch----
            venue  freq
0     Pizza Place  0.22
1             Pub  0.11
2        Pharmacy  0.11
3             Gym  0.11
4  Sandwich Place  0.11


----Bathurst Manor / Wilson Heights / Downsview North----
                       venue  freq
0                       Bank  0.10
1                Coffee Shop  0.10
2                Bridal Shop  0.05
3              Shopping Mall  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4        Movie Theater  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1      Sandwich Place  

4  Mexican Restaurant  0.04


----Richmond / Adelaide / King----
           venue  freq
0    Coffee Shop  0.08
1           Café  0.05
2     Restaurant  0.04
3            Gym  0.04
4  Deli / Bodega  0.03


----Rosedale----
                venue  freq
0                Park  0.50
1          Playground  0.25
2               Trail  0.25
3   Accessories Store  0.00
4  Mexican Restaurant  0.00


----Roselawn----
                      venue  freq
0                    Garden   1.0
1         Accessories Store   0.0
2            Massage Studio   0.0
3            Medical Center   0.0
4  Mediterranean Restaurant   0.0


----Rouge Hill / Port Union / Highland Creek----
                        venue  freq
0  Construction & Landscaping  0.33
1              History Museum  0.33
2                         Bar  0.33
3           Accessories Store  0.00
4          Miscellaneous Shop  0.00


----Runnymede / Swansea----
                venue  freq
0                Café  0.08
1         Coffee Shop  0.08
2     

In [170]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [171]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                          Agincourt                Lounge   
1                            Alderwood / Long Branch           Pizza Place   
2  Bathurst Manor / Wilson Heights / Downsview North           Coffee Shop   
3                                    Bayview Village                  Bank   
4                 Bedford Park / Lawrence Manor East           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue      4th Most Common Venue  \
0        Breakfast Spot          Skating Rink  Latin American Restaurant   
1                   Gym                  Pool               Skating Rink   
2                  Bank           Gas Station         Frozen Yogurt Shop   
3    Chinese Restaurant   Japanese Restaurant                       Café   
4            Restaurant        Sandwich Place         Italian Restaurant   

  5th Most Common Venue  
0        Clothing Store  
1           Coffee Shop  
2            Restaurant  
3           Yoga Studio  
4              Boutique

In [172]:
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [199]:
toronto_merged = dfc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged = toronto_merged.dropna()


toronto_merged.head()

Postal code      Borough                              Neighborhood  \
0         M1B  Scarborough                           Malvern / Rouge   
1         M1C  Scarborough  Rouge Hill / Port Union / Highland Creek   
2         M1E  Scarborough       Guildwood / Morningside / West Hill   
3         M1G  Scarborough                                    Woburn   
4         M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude       1st Most Common Venue  \
0  43.806686 -79.194353        Fast Food Restaurant   
1  43.784535 -79.160497  Construction & Landscaping   
2  43.763573 -79.188711          Mexican Restaurant   
3  43.770992 -79.216917                 Coffee Shop   
4  43.773136 -79.239476            Hakka Restaurant   

        2nd Most Common Venue        3rd Most Common Venue  \
0  Construction & Landscaping  Eastern European Restaurant   
1                         Bar               History Museum   
2              Breakfast Spot          Rental Car Location   
3           Korean Restaurant                  Yoga Studio   
4                 Gas Station          Fried Chicken Joint   

  4th Most Common Venue        5th Most Common Venue  
0   Distribution Center                      Dog Run  
1           Yoga Studio  Eastern European Restaurant  
2                  Bank            Electronics Store  
3   Dumpling Restaurant          Distribution Center  
4                  Bank                       Bakery

In [208]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'ClusterLabels'

In [209]:
toronto_merged.loc[toronto_merged['ClusterLabels'] ==1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

KeyError: 'ClusterLabels'